# MinCost Flow A caterer’s problem
[Murwan Siddig](mailto:msiddig@clemson.edu)

**Problem setting:**
* Demand for napkins is 100 on weekdays and 125 on weekends
* Cost of new napkins: 10 cents per napkin
* 2-day laundry: 1 cent per napkin
* 1-day laundry: 2 cents per napkin

**Additional assumptions:**
* There is no initial inventory of napkins
* At the end, no clean napkins remain
-----------------------------------------------------------
**Goal: Minimize the cost of meeting demand**

In [1]:
#technical lines 
using LightGraphs

In [8]:
# Specify original network structure, arc costs, capacity, and netSupply 
adj = zeros(Int64,14,14);
arccost = zeros(Int64,14,14);
arccap = zeros(Int64,14,14);
for i=2:7
    adj[1,i] = 1;
    arccost[1,i] = 10;
    if i >= 6
        arccap[1,i] = 125;
    else
        arccap[1,i] = 100;
    end
end

for i=8:12
    adj[i,i-6] = 1;
    arccost[i,i-6] = 2;
    arccap[i,i-6] = 100;
    adj[i,i-5] = 1;
    arccost[i,i-5] = 1;
    arccap[i,i-5] = 100;
    adj[i,14] = 1;
    arccap[i,14] = 100;
end
adj[13,7] = 1;
arccost[13,7] = 2;
arccap[13,7] = 125;
adj[13,14] = 1;
arccap[13,14] = 125;
adj[14,1] = 1;
arccap[14,1] = 625;

flow = zeros(Int64,14,14);
netSupply = zeros(Int64, 14);
netSupply[1] = 25;
for i = 2:5
    netSupply[i] = -100;
end
netSupply[6] = -125;
netSupply[7] = -125;
for i = 8:12
    netSupply[i] = 100;
end
netSupply[13] = 125;

In [9]:
function constructResidualNetwork(adj, flow, arccap)
    n = size(adj)[1];
    adj2 = zeros(Int64,n,n);
    for i = 1:n
        for j = 1:n
            if adj[i,j] == 1
                if flow[i,j] > 1e-5
                    adj2[j,i] = 1;
                end
                if flow[i,j] < arccap[i,j]-1e-5
                    adj2[i,j] = 1;
                end
            end
        end
    end         
    residualNetwork = DiGraph(adj2);
    return residualNetwork;
end

constructResidualNetwork (generic function with 1 method)

In [10]:
function successiveSP(adj,arccost,arccap,flow,netSupply)
    mincost = 0;
    n = size(adj)[1];
    nodePotential = zeros(Int64,n);
    deficitFlag = 1;
    nIter = 0;
    while (deficitFlag == 1)
        nIter = nIter+1;
        deficitFlag = 0;
        source = -1;
        maxSupply = 0;
        for i = 1:n
            if netSupply[i] > maxSupply + 1e-5
                source = i;
                maxSupply = netSupply[i];
            end
        end
        println("maxSupply = ", maxSupply);
        if source != -1
            deficitFlag = 1;
            # begin loop, find the shortest path from source to all other nodes
            # First construct residual network and the residual capacity according to flow
            residualNetwork = constructResidualNetwork(adj, flow, arccap);
            residualCap = zeros(Int64,n,n);
            for i = 1:n
                for j = 1:n
                    if adj[i,j] == 1
                        residualCap[i,j] = arccap[i,j] - flow[i,j];
                        if flow[i,j] > 1e-5
                            residualCap[j,i] = flow[i,j];
                        end
                    end
                end
            end
            reducedArcCost = zeros(Int64,n,n);
            for i = 1:n
                for j = 1:n
                    if adj[i,j] == 1
                        if flow[i,j] < arccap[i,j]-1e-5
                            # Forward arc
                            reducedArcCost[i,j] = arccost[i,j] - nodePotential[i] + nodePotential[j];
                        end
                        if flow[i,j] > 1e-5
                            # Backward arc
                            reducedArcCost[j,i] = -arccost[i,j] - nodePotential[j] + nodePotential[i];
                        end
                    end
                end
            end       
            
            println("reducedArcCost = ", reducedArcCost);
            
            # Get both distance from source to each other node, and precedence
            dijk=dijkstra_shortest_paths(residualNetwork, source, reducedArcCost);
            #dijk = bellman_ford_shortest_paths(residualNetwork, source, reducedArcCost);
            # find a sink node with a deficit
            sink = -1;
            for i = 1:n
                if dijk.parents[i] != 0 && netSupply[i] < -1e-5
                    sink = i;
                    break;
                end
            end
            if sink == -1
                println("Cannot find a sink, something must be wrong!");
                exit(0);
            end
            # Backtrack to source and find a shortest path
            path = Any[];
            iter = sink;
            while iter != 0
                push!(path, iter);
                iter = dijk.parents[iter];
            end
            println("path = ", path);
            # Now have a path, first find the bottleneck on the maximum amount we can augment the flow
            bottleneck = 1e8;
            nPath = length(path);
            for i = 1:(nPath-1)
                ii = path[nPath-i+1];
                jj = path[nPath-i];
                if residualCap[ii,jj] < bottleneck
                    bottleneck = residualCap[ii,jj];
                end
            end
            if netSupply[source] < bottleneck
                bottleneck = netSupply[source];
            end
            if -netSupply[sink] < bottleneck
                bottleneck = -netSupply[sink];
            end
            
            # double check
            if bottleneck < 1e-5
                println("something must be wrong, bottleneck should be > 0!")
                exit(0);
            end
            # Now augment flow along the path by amount: bottleneck
            netSupply[source] -= bottleneck;
            netSupply[sink] += bottleneck;
            for i = 1:(nPath-1)
                ii = path[nPath-i+1];
                jj = path[nPath-i];
                # decide if (ii,jj) is a forward or a backward arc
                if adj[ii,jj] == 1
                    # forward arc
                    flow[ii,jj] += bottleneck;
                else
                    # backward arc: push flow back
                    flow[jj,ii] -= bottleneck;
                end
            end
            
            # update node potentials
            for i = 1:n
                if dijk.dists[i] < 1e5
                    nodePotential[i] = nodePotential[i]-dijk.dists[i];
                else
                    nodePotential[i] = nodePotential[i]-1e5;
                end
            end
        end
        println("nIter = ", nIter);
    end
    for i = 1:n
        for j = 1:n
            mincost += flow[i,j]*arccost[i,j];
        end
    end
    return mincost,flow;
end

successiveSP (generic function with 1 method)

In [11]:
mincost, flow = successiveSP(adj,arccost,arccap,flow,netSupply);
println("===========================================================")
println("===========================================================")
println("mincost = ", mincost)
for i=1:size(flow)[1]
    println("flow = ", flow[i,:])
end

maxSupply = 125
reducedArcCost = [0 10 10 10 10 10 10 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 2 1 0 0 0 0 0 0 0 0 0 0 0; 0 0 2 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 2 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 2 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 2 1 0 0 0 0 0 0 0; 0 0 0 0 0 0 2 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
path = Any[2, 1, 14, 13]
nIter = 1
maxSupply = 100
reducedArcCost = [0 0 0 0 0 0 8 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 99992 99991 0 0 0 0 0 0 0 0 0 0 100000; 0 0 99992 99991 0 0 0 0 0 0 0 0 0 100000; 0 0 0 99992 99991 0 0 0 0 0 0 0 0 100000; 0 0 0 0 99992 99991 0 0 0 0 0 0 0 100000; 0 0 0 0 0 99992 99999 0 0 0 0 0 0 100000; 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
path = Any[3, 